In [62]:
import math
import random
import numpy as np





class trash:

    def __init__(self,r_val) -> None:
        
        self.discard_deck = []
        self.draw_deck = []
        self.player1 = {'deck':[], 'array_count': [10]}
        self.player2 = {'deck':[], 'array_count': [10]}
        

        self.num_turns = 0
        self.transitions = 0
        self.winner = []
        self.game_over = False
        self.deck = self.build_deck()
        self.deck = self.shuffle_cards(self.deck)
        self.initial_set_decks()

    def build_deck(self):

        deck = []
        num_cards = 4
        c = 0
        for _ in (range(13)):
            #print(c)
            #add 4 of every card to deck
            deck.extend([c+1] * num_cards)

            c = c+1
        return deck

    def shuffle_cards(self,deck:list):

        #Fisher-Yates for shuffling

       
        n = len(deck)

        for c in range(n):
            r = random.uniform(.01,.999) #works for now
            p = int((r*(n-c))+c)
            #print(p)
            #print(c)

            #constant time complexity for this operation
            deck[c], deck[p] = deck[p], deck[c]

            #c = c+1
        return deck 

    #SETTRS
    def initial_set_decks(self):
        self.player1['deck'].extend(self.deck[11:22])
        self.player2['deck'].extend(self.deck[0:11])
        self.draw_deck.extend(self.deck[22:51])
        self.discard_deck.append(self.deck[51])
    
    
    def can_place_card(self, card, array):
        # Jacks are wild cards and can be placed anywhere
        if card == 11:  # Jack is represented by 11
            return True
        
        # Check if the slot for the card is empty
        return array[card-1] == 'X'
    
    def place_card(self, card, array):
        # If the card is a Jack, find the first empty slot
        if card == 11:  # Jack is represented by 11
            index = array.index('X')
        else:
            index = card - 1

        # Swap the card in the array and return the old card
        old_card = array[index]
        array[index] = card
        return old_card

    def is_array_cleared(self, array):
        return all(isinstance(slot, int) for slot in array)  # All slots are filled with numbers

    def is_game_over(self):
        # The game is over when either player's array_count is empty.
        return self.player1['array_count'] == 0 or self.player2['array_count'] == 0

    def play_turn(self, player, opponent):
        if not self.draw_deck and len(self.discard_deck) > 1:
            self.draw_deck = self.shuffle_cards(self.discard_deck[:-1])
            self.discard_deck = [self.discard_deck[-1]]

        # Player chooses a card from the draw pile
        card_in_hand = self.draw_deck.pop()

        array = player['deck']

        while self.can_place_card(card_in_hand, array):
            card_in_hand = self.place_card(card_in_hand, array)
            
            # Queens and Kings are trash cards
            if card_in_hand in [12, 13]:  # Queen and King are represented by 12 and 13
                break

        # Place the card in discard pile at the end of the turn
        self.discard_deck.append(card_in_hand)

        # Check if player has cleared their array
        if self.is_array_cleared(player['deck']):
            player['array_count'].pop()
            if not self.is_game_over():
                next_array_size = player['array_count'][-1] if player['array_count'] else 0
                player['deck'] = ['X'] * next_array_size
                self.shuffle_and_reset_deck(player)

        # Check winning transitions
        if len(player['array_count']) < len(opponent['array_count']):
            self.transitions += 1

    def shuffle_and_reset_deck(self, current_player):
        # This function resets the game state by shuffling all cards and dealing new arrays.
        all_cards = current_player['deck'] + self.draw_deck + self.discard_deck
        shuffled_cards = self.shuffle_cards(all_cards)

        # Reset the current player's deck
        next_array_count = current_player['array_count'][0] - 1  # Decrease the size of the array for the next round
        current_player['deck'] = ['X'] * next_array_count
        current_player['array_count'].insert(0, next_array_count)

        # Update the draw and discard piles
        self.draw_deck = shuffled_cards[next_array_count:]
        self.discard_deck = [self.draw_deck.pop()]  # Take the top card from draw pile and put it in the discard pile


    def play_game(self):
        while not self.is_game_over():
            self.play_turn(self.player1, self.player2)
            self.num_turns += 1

            if self.is_game_over():  # check if the game ended after player1's turn
                break
            
            self.play_turn(self.player2, self.player1)
            self.num_turns += 1


        # Determine the winner
        if len(self.player1['array_count']) == 0:
            self.winner = "Player 1"
        else:
            self.winner = "Player 2"



In [63]:
trash_sim = trash(.2)

#Consistency check
#print(len(trash_sim.discard_deck) + len(trash_sim.draw_deck) + len(trash_sim.player1['deck'])+ len(trash_sim.player2['deck']))
#print(len(trash_sim.discard_deck) )

game = trash(1)
print(game.player1['array_count'])
game.play_game()
print(f"The winner is {game.winner} after {game.num_turns} turns with {game.transitions} transitions!")


[10]


ValueError: 'X' is not in list